<div>
<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=220 height=220 align=left class="saturate">

<br>
<font face="Times New Roman">
<div dir=ltr align=center> 
<!-- <font color=0F5298 size=7> -->
<font color=0F5298 size=6>
    Introduction to Machine Learning <br> <br>
<!-- <font color=2565AE size=5> -->
<font size=5>
    Computer Engineering Department <br>
    Spring 2023 <br> <br>
<font color=606060 size=5>
    Homework 2: Practical - LDA & QDA <br> <br>
<font color=686880 size=4>
    TAs: Arman Malekzadeh - Arshia Soltani Moakhar
    
____

### Full Name : Mohammad Abolnejadian
### Student Number : 98103867
___

In [1152]:
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as SklearnLDA
from sklearn.metrics import classification_report
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as SklearnQDA
from sklearn.metrics import roc_auc_score

# Exploration (20 points)

In [1153]:
!wget https://github.com/sharif-ml/sharif-ml.github.io/raw/main/assignments/dataset-train.csv

--2023-04-20 19:19:37--  https://github.com/sharif-ml/sharif-ml.github.io/raw/main/assignments/dataset-train.csv
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sharif-ml/sharif-ml.github.io/main/assignments/dataset-train.csv [following]
--2023-04-20 19:19:37--  https://raw.githubusercontent.com/sharif-ml/sharif-ml.github.io/main/assignments/dataset-train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89401 (87K) [text/plain]
Saving to: ‘dataset-train.csv.42’

dataset-train.csv.4 100%[===================>]  87.31K  --.-KB/s    in 0.02s   

2023-04-20 19:19:37 (5.18 MB/s) - ‘dataset-train.

Load the dataset as a dataframe

In [1154]:
df = pd.read_csv("dataset-train.csv")
df

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Country,Timestamp,Clicked on Ad,Gender
0,68.95,35,61833.90,256.09,Cloned 5thgeneration orchestration,Wrightburgh,Tunisia,2016-03-27 00:53:11,0,female
1,80.23,31,68441.85,193.77,Monitored national standardization,West Jodi,Nauru,2016-04-04 01:39:02,0,male
2,69.47,26,59785.94,236.50,Organic bottom-line service-desk,Davidton,San Marino,2016-03-13 20:35:42,0,female
3,74.15,29,54806.18,245.89,Triple-buffered reciprocal time-frame,West Terrifurt,Italy,2016-01-10 02:31:19,0,male
4,68.37,35,73889.99,225.58,Robust logistical utilization,South Manuel,Iceland,2016-06-03 03:36:18,0,female
...,...,...,...,...,...,...,...,...,...,...
795,75.64,29,69718.19,204.82,Pre-emptive well-modulated moderator,West Rhondamouth,Iran,2016-03-15 19:35:19,0,male
796,79.22,27,63429.18,198.79,Intuitive modular system engine,North Debra,Belgium,2016-03-11 12:39:19,0,male
797,77.05,34,65756.36,236.08,Centralized value-added hierarchy,Villanuevastad,Namibia,2016-05-17 18:06:46,0,female
798,66.83,46,77871.75,196.17,Assimilated hybrid initiative,North Jeremyport,Cyprus,2016-02-28 23:10:32,0,male


Get the minimum "Area Income" for all Italians

In [1155]:
min_area_income_italians = df[df["Country"].str.lower() == 'italy'].min()['Area Income']
min_area_income_italians

54286.1

Make a new dataframe called "sample_df" by sampling 100 rows from the previously made dataframe

In [1156]:
sample_df = df.sample(100)
sample_df

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Country,Timestamp,Clicked on Ad,Gender
362,79.72,28,66025.11,193.80,Configurable tertiary budgetary management,Wrightburgh,Albania,2016-05-20 12:17:59,0,male
76,49.81,35,57009.76,120.06,Seamless real-time array,Ramirezhaven,Faroe Islands,2016-01-05 04:18:46,1,male
351,73.71,23,56605.12,211.38,Inverse local hub,New Sheila,El Salvador,2016-02-15 05:35:54,0,male
525,66.14,41,78092.95,165.27,Re-engineered impactful software,North Isabellaville,Western Sahara,2016-03-23 08:52:31,0,female
185,46.88,54,43444.86,136.64,Virtual scalable secured line,Port Melissaberg,Singapore,2016-02-04 08:53:37,1,female
...,...,...,...,...,...,...,...,...,...,...
248,42.44,56,43450.11,168.27,Profit-focused attitude-oriented task-force,Petersonfurt,Tajikistan,2016-02-10 15:23:17,1,female
466,82.38,35,25603.93,159.60,Polarized analyzing intranet,Port Blake,Spain,2016-07-18 01:36:37,1,female
407,48.73,27,34191.23,142.04,Versatile content-based protocol,Monicaview,Bangladesh,2016-07-09 16:23:33,1,female
541,75.65,39,64021.55,247.90,Organized upward-trending contingency,Brianabury,Bahrain,2016-05-30 02:34:25,0,male


Compare all men and women aged less than 30 years in terms of their average "Daily Internet Usage". Your result should contain the average daily usage for each age and gender, seperately.

In [1157]:
men_df = df[df['Gender'].str.lower() == 'male']
women_df = df[df['Gender'].str.lower() == 'female']
ages_below_thirty = sorted(filter(lambda x: x<30, df['Age'].unique()))
compare_df = pd.DataFrame(columns = ['Men', 'Women'])
for age in ages_below_thirty:
  average_men = men_df[men_df['Age'] == age]['Daily Internet Usage'].mean()
  average_women = women_df[women_df['Age'] == age]['Daily Internet Usage'].mean()
  compare_df.loc[age] = [round(average_men, 2), round(average_women, 2)]
compare_df

,Men,Women
19,189.52,184.94
20,198.98,202.16
21,212.82,186.98
22,188.84,200.07
23,184.62,187.11
24,200.06,178.69
25,206.53,193.62
26,203.92,192.74
27,205.92,189.16
28,196.42,205.87


# Preprocessing (30 points)

Remove the following columns: "Ad Topic Line", "Timestamp"

In [1158]:
preprocess_df = df.drop(columns = ['Ad Topic Line', 'Timestamp'])
preprocess_df

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,City,Country,Clicked on Ad,Gender
0,68.95,35,61833.90,256.09,Wrightburgh,Tunisia,0,female
1,80.23,31,68441.85,193.77,West Jodi,Nauru,0,male
2,69.47,26,59785.94,236.50,Davidton,San Marino,0,female
3,74.15,29,54806.18,245.89,West Terrifurt,Italy,0,male
4,68.37,35,73889.99,225.58,South Manuel,Iceland,0,female
...,...,...,...,...,...,...,...,...
795,75.64,29,69718.19,204.82,West Rhondamouth,Iran,0,male
796,79.22,27,63429.18,198.79,North Debra,Belgium,0,male
797,77.05,34,65756.36,236.08,Villanuevastad,Namibia,0,female
798,66.83,46,77871.75,196.17,North Jeremyport,Cyprus,0,male


Convert the "City" and "Country" columns to their one-hot encoded versions.
For each set of one-hot encoded columns, remove one of them to make the columns not correlated. Finally, remove the original columns named "City" and "Country".

In [1159]:
one_hoted_df = pd.get_dummies(preprocess_df, columns=['City', 'Country'], prefix = ['City', 'Country'])
one_hoted_df

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Clicked on Ad,Gender,City_Adamsbury,City_Alanview,City_Alexanderfurt,City_Alexanderview,...,Country_Uruguay,Country_Uzbekistan,Country_Vanuatu,Country_Venezuela,Country_Vietnam,Country_Wallis and Futuna,Country_Western Sahara,Country_Yemen,Country_Zambia,Country_Zimbabwe
0,68.95,35,61833.90,256.09,0,female,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,80.23,31,68441.85,193.77,0,male,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,69.47,26,59785.94,236.50,0,female,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,74.15,29,54806.18,245.89,0,male,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,68.37,35,73889.99,225.58,0,female,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,75.64,29,69718.19,204.82,0,male,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
796,79.22,27,63429.18,198.79,0,male,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
797,77.05,34,65756.36,236.08,0,female,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
798,66.83,46,77871.75,196.17,0,male,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [1160]:
selected_city = random.choice(preprocess_df['City'].unique())
selected_country = random.choice(preprocess_df['Country'].unique())
selected_city, selected_country

('North Garyhaven', 'Saint Martin')

In [1161]:
one_hoted_df = one_hoted_df.drop(columns = ['City_' + selected_city, 'Country_'+ selected_country])

In [1162]:
one_hoted_df

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Clicked on Ad,Gender,City_Adamsbury,City_Alanview,City_Alexanderfurt,City_Alexanderview,...,Country_Uruguay,Country_Uzbekistan,Country_Vanuatu,Country_Venezuela,Country_Vietnam,Country_Wallis and Futuna,Country_Western Sahara,Country_Yemen,Country_Zambia,Country_Zimbabwe
0,68.95,35,61833.90,256.09,0,female,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,80.23,31,68441.85,193.77,0,male,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,69.47,26,59785.94,236.50,0,female,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,74.15,29,54806.18,245.89,0,male,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,68.37,35,73889.99,225.58,0,female,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,75.64,29,69718.19,204.82,0,male,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
796,79.22,27,63429.18,198.79,0,male,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
797,77.05,34,65756.36,236.08,0,female,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
798,66.83,46,77871.75,196.17,0,male,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Convert "Gender" column to a new binary column (having only 0 and 1 values). Then remove the original column.

In [1163]:
one_hoted_df = pd.get_dummies(one_hoted_df, columns = ['Gender'], prefix = ['Gender'])

In [1164]:
one_hoted_df = one_hoted_df.drop(columns = ['Gender_female'])

In [1165]:
#1s are male and 0s are female
one_hoted_df = one_hoted_df.rename(columns = {'Gender_male': 'Gender'})
one_hoted_df

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Clicked on Ad,City_Adamsbury,City_Alanview,City_Alexanderfurt,City_Alexanderview,City_Alexisland,...,Country_Uzbekistan,Country_Vanuatu,Country_Venezuela,Country_Vietnam,Country_Wallis and Futuna,Country_Western Sahara,Country_Yemen,Country_Zambia,Country_Zimbabwe,Gender
0,68.95,35,61833.90,256.09,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,80.23,31,68441.85,193.77,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,69.47,26,59785.94,236.50,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,74.15,29,54806.18,245.89,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,68.37,35,73889.99,225.58,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,75.64,29,69718.19,204.82,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
796,79.22,27,63429.18,198.79,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
797,77.05,34,65756.36,236.08,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
798,66.83,46,77871.75,196.17,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


By dropping the "Clicked on Ad" column, make a new dataframe called "features_df"

In [1166]:
features_df = one_hoted_df.drop(columns = ['Clicked on Ad'])
features_df

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,City_Adamsbury,City_Alanview,City_Alexanderfurt,City_Alexanderview,City_Alexisland,City_Aliciatown,...,Country_Uzbekistan,Country_Vanuatu,Country_Venezuela,Country_Vietnam,Country_Wallis and Futuna,Country_Western Sahara,Country_Yemen,Country_Zambia,Country_Zimbabwe,Gender
0,68.95,35,61833.90,256.09,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,80.23,31,68441.85,193.77,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,69.47,26,59785.94,236.50,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,74.15,29,54806.18,245.89,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,68.37,35,73889.99,225.58,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,75.64,29,69718.19,204.82,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
796,79.22,27,63429.18,198.79,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
797,77.05,34,65756.36,236.08,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
798,66.83,46,77871.75,196.17,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


Make a new dataframe called "target_df" containing only the "Clicked on Ad" column

In [1167]:
target_df = preprocess_df.loc[:, ["Clicked on Ad"]]
target_df

,Clicked on Ad
0,0
1,0
2,0
3,0
4,0
...,...
795,0
796,0
797,0
798,0


Make two numpy arrays called "x" and "y" from "features_df" and "target_df" dataframes, respectively.

In [1168]:
x = features_df.to_numpy()
y = target_df.to_numpy()

# Training Models (40 points)

Split the dataset (`x` and `y`) into "training" and "testing" parts containing 80% and 20% of the data respectively.

In [1169]:
x_train, x_test, y_train, y_test = train_test_split(x,y , train_size=0.8)

Train both LDA and QDA models on the data using `sklearn`

In [1170]:
lda_sklearn = SklearnLDA()
lda_sklearn.fit(x_train, np.ravel(y_train))

LinearDiscriminantAnalysis()

In [1171]:
qda_sklearn = SklearnQDA()
qda_sklearn.fit(x_train, np.ravel(y_train))

/usr/local/lib/python3.9/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QuadraticDiscriminantAnalysis()

Train LDA and QDA using your own code!

Linear Discriminant Analysis

In [1172]:
class LinearDiscriminantAnalysis:
    """
    Fits it on data, then uses predict to get results.
    """

    def __init__(self):
        pass

    
    def fit(self, X, y):
        """Fit the training data
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            Features
        y : array-like, shape = [n_samples, n_labels]
            Labels
        Returns
        -------
        self : object
        FOR TWO CLASSES
        """

        self.n_samples, self.n_features = X.shape
        X0 = X[y==0]
        X1 = X[y==1]
        Xs = [X0, X1]
        # Get prior probabilities 
        self.prior_prob0 = len(X0)/self.n_samples
        self.prior_prob1 = len(X1)/self.n_samples
        # Get mean of each category
        self.mu0 = X0.mean(0).reshape(self.n_features, 1)
        self.mu1 = X1.mean(0).reshape(self.n_features, 1)
        # Get scatter
        self.Sigma = np.zeros((self.n_features, self.n_features))  
        for x in X0:
          x = x.reshape(-1,1)
          self.Sigma += np.dot((x - self.mu0), (x - self.mu0).T)
        for x in X1:
          x = x.reshape(-1,1)
          self.Sigma += np.dot((x - self.mu1), (x - self.mu1).T)
        # Normalize
        self.Sigma /= self.n_samples
        
    def _calculate_x_given_y(self, x, mu_k, Sigma):
        #Normalize sigma
        Sigma += np.eye(self.n_features) * 0.6
        ans = np.exp(-(1/2)*(x - mu_k).T @ np.linalg.inv(Sigma) @ (x - mu_k))
        return ans
        

    def predict(self, X):
        """ Predicts the value after the model has been trained.
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            Features
        Returns
        -------
        Predicted value
        """
        n_samples, n_features = X.shape
        '''
        Write your own code
        '''
        classes = [0, 1]
        y_n = np.empty(len(X))

        for i, x_n in enumerate(X):
            
            x_n = x_n.reshape(-1, 1)
            prob_classes = np.empty(2)
            # Calculate for class 0
            p_x_given_y = self._calculate_x_given_y(x_n, self.mu0, self.Sigma)
            p_y_given_x = self.prior_prob0*p_x_given_y
            prob_classes[0] = p_y_given_x
            # Calculate for class 1
            p_x_given_y = self._calculate_x_given_y(x_n, self.mu1, self.Sigma)
            p_y_given_x = self.prior_prob1*p_x_given_y
            prob_classes[1] = p_y_given_x
            
            y_n[i] = classes[np.argmax(prob_classes)]
        
        return y_n


Quadratic Discriminant Analysis (Optional) (15 bonus points)

In [1173]:
class QuadraticDiscriminantAnalysis:
    """
    Fits it on data, then uses predict to get results.
    """

    def __init__(self):
        pass
    
    def fit(self, X, y):
        """Fit the training data
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            Features
        y : array-like, shape = [n_samples, n_labels]
            Labels
        Returns
        -------
        self : object
        FOR 2 CLASSES
        """

        self.n_samples, self.n_features = X.shape
        '''
        Write your own code
        '''
        X0 = X[y==0]
        X1 = X[y==1]
        Xs = [X0, X1]
        # Get prior probabilities 
        self.prior_prob0 = len(X0)/self.n_samples
        self.prior_prob1 = len(X1)/self.n_samples
        # Get mean of each category
        self.mu0 = X0.mean(0).reshape(self.n_features, 1)
        self.mu1 = X1.mean(0).reshape(self.n_features, 1)
        # Get scatter
        self.Sigma0 = np.zeros((self.n_features, self.n_features)) 
        self.Sigma1 = np.zeros((self.n_features, self.n_features))  
        for x in X0:
          x = x.reshape(-1,1)
          self.Sigma0 += np.dot((x - self.mu0), (x - self.mu0).T)
        self.Sigma0 = self.Sigma0 / len(X0)
        for x in X1:
          x = x.reshape(-1,1)
          self.Sigma1 += np.dot((x - self.mu1), (x - self.mu1).T)
        self.Sigma1 = self.Sigma1 / len(X1)
        

         
    def _calculate_x_given_y(self, x, mu_k, Sigma_k):
        Sigma_k += np.eye(self.n_features) * 0.6
        ans = np.linalg.det(Sigma_k)**(-1/2) * np.exp(-(1/2)*(x - mu_k).T @ np.linalg.inv(Sigma_k) @ (x - mu_k))
        return ans
    
    def predict(self, X):
        """ Predicts the value after the model has been trained.
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            Features
        Returns
        -------
        Predicted value
        """
        n_samples, n_features = X.shape
        '''
        Write your own code
        '''
        classes = [0, 1]
        y_n = np.empty(len(X))

        for i, x_n in enumerate(X):
            
            x_n = x_n.reshape(-1, 1)
            prob_classes = np.empty(2)
            # Calculate for class 0
            p_x_given_y = self._calculate_x_given_y(x_n, self.mu0, self.Sigma0)
            p_y_given_x = self.prior_prob0*p_x_given_y
            prob_classes[0] = p_y_given_x
            # Calculate for class 1
            p_x_given_y = self._calculate_x_given_y(x_n, self.mu1, self.Sigma1)
            p_y_given_x = self.prior_prob1*p_x_given_y
            prob_classes[1] = p_y_given_x
            
            y_n[i] = classes[np.argmax(prob_classes)]
        
        return y_n

In [1174]:
# Train your own models
lda_self = LinearDiscriminantAnalysis()
lda_self.fit(x_train, np.ravel(y_train))
qda_self = QuadraticDiscriminantAnalysis()
qda_self.fit(x_train, np.ravel(y_train))

# Evaluation (10 points)

Using all 4 models, predict the labels for all samples belonging to the "testing" part

In [1175]:
y_pred_sklda = lda_sklearn.predict(x_test)
y_pred_lda = lda_self.predict(x_test)

In [1176]:
y_pred_skqda =  qda_sklearn.predict(x_test)
y_pred_qda = qda_self.predict(x_test)

/usr/local/lib/python3.9/dist-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


Calculate Accuracy, Recall, and F1-Scores

**For LDA**

In [1177]:
print(f"Sklearn LDA:\n${classification_report(y_test, y_pred_sklda)}")

Sklearn LDA:
$              precision    recall  f1-score   support

           0       0.82      0.87      0.84        86
           1       0.84      0.77      0.80        74

    accuracy                           0.82       160
   macro avg       0.83      0.82      0.82       160
weighted avg       0.83      0.82      0.82       160



In [1178]:
print(f"Implemented LDA:\n${classification_report(y_test, y_pred_lda)}")

Implemented LDA:
$              precision    recall  f1-score   support

           0       0.93      0.99      0.96        86
           1       0.99      0.92      0.95        74

    accuracy                           0.96       160
   macro avg       0.96      0.95      0.96       160
weighted avg       0.96      0.96      0.96       160



**For QDA**

In [1179]:
print(f"Sklearn QDA:\n${classification_report(y_test, y_pred_skqda)}")

Sklearn QDA:
$              precision    recall  f1-score   support

           0       0.92      0.94      0.93        86
           1       0.93      0.91      0.92        74

    accuracy                           0.93       160
   macro avg       0.93      0.92      0.92       160
weighted avg       0.93      0.93      0.92       160



In [1180]:
print(f"Implemented QDA:\n${classification_report(y_test, y_pred_qda)}")

Implemented QDA:
$              precision    recall  f1-score   support

           0       0.54      1.00      0.70        86
           1       1.00      0.01      0.03        74

    accuracy                           0.54       160
   macro avg       0.77      0.51      0.36       160
weighted avg       0.75      0.54      0.39       160



Compute the ROC AUC Score using `sklearn` (for both LDA and QDA models trained using this library)

In [1181]:
roc_auc_score(y_test, y_pred_sklda)

0.8211816467630421

In [1182]:
roc_auc_score(y_test, y_pred_lda)

0.9536455059710873

In [1183]:
roc_auc_score(y_test, y_pred_qda)

0.5067567567567568

In [1184]:
roc_auc_score(y_test, y_pred_skqda)

0.9236329352608421

# Kaggle Competition

Link: https://www.kaggle.com/t/988b0aa2db2446fd85cff571d69c6386